In [17]:
import pandas as pd
import numpy
import sklearn
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt


train_complete = pd.DataFrame(pd.read_csv('happiness_train_complete.csv',encoding = 'gbk'))
test_complete = pd.DataFrame(pd.read_csv('happiness_test_complete.csv',encoding ='gbk'))
#data = train_complete.append(test_complete)
#data.info(verbose=True,null_counts=True)
##data.head()
data_full = train_complete.append(test_complete)
pd.set_option('display.max_rows', None)#显示全部行
pd.set_option('display.max_columns', None)#显示全部列

In [18]:
(data_full.shape,train_complete.shape,test_complete .shape)

((10968, 140), (8000, 140), (2968, 139))

In [19]:
y_labels = data_full.loc[:,'happiness']
y_labels=y_labels.dropna(axis=0, inplace=False)
print(y_labels.shape)
y_labels.value_counts()

(8000,)


 4.0    4818
 5.0    1410
 3.0    1159
 2.0     497
 1.0     104
-8.0      12
Name: happiness, dtype: int64

In [20]:
#将-8换成3
y_labels=y_labels.map(lambda x:3 if x==-8 else x)

In [21]:
data_full = data_full.drop(['happiness'],axis = 1)
data_full.info(verbose = True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10968 entries, 0 to 2967
Data columns (total 139 columns):
birth                   10968 non-null int64
car                     10968 non-null int64
city                    10968 non-null int64
class                   10968 non-null int64
class_10_after          10968 non-null int64
class_10_before         10968 non-null int64
class_14                10968 non-null int64
county                  10968 non-null int64
daughter                10968 non-null int64
depression              10968 non-null int64
edu                     10968 non-null int64
edu_other               6 non-null object
edu_status              9399 non-null float64
edu_yr                  8212 non-null float64
equity                  10968 non-null int64
f_birth                 10968 non-null int64
f_edu                   10968 non-null int64
f_political             10968 non-null int64
f_work_14               10968 non-null int64
family_income           10967 non-nul

In [22]:
data_full.isnull().sum()

birth                       0
car                         0
city                        0
class                       0
class_10_after              0
class_10_before             0
class_14                    0
county                      0
daughter                    0
depression                  0
edu                         0
edu_other               10962
edu_status               1569
edu_yr                   2756
equity                      0
f_birth                     0
f_edu                       0
f_political                 0
f_work_14                   0
family_income               1
family_m                    0
family_status               0
floor_area                  0
gender                      0
health                      0
health_problem              0
height_cm                   0
house                       0
hukou                       0
hukou_loc                   4
id                          0
inc_ability                 0
inc_exp                     0
income    

In [23]:
data = data_full.drop(['invest_other','edu_other','property_other'],axis = 1)
data.isnull().sum()

birth                      0
car                        0
city                       0
class                      0
class_10_after             0
class_10_before            0
class_14                   0
county                     0
daughter                   0
depression                 0
edu                        0
edu_status              1569
edu_yr                  2756
equity                     0
f_birth                    0
f_edu                      0
f_political                0
f_work_14                  0
family_income              1
family_m                   0
family_status              0
floor_area                 0
gender                     0
health                     0
health_problem             0
height_cm                  0
house                      0
hukou                      0
hukou_loc                  4
id                         0
inc_ability                0
inc_exp                    0
income                     0
insur_1                    0
insur_2       

In [24]:
data.isnull().sum()

birth                      0
car                        0
city                       0
class                      0
class_10_after             0
class_10_before            0
class_14                   0
county                     0
daughter                   0
depression                 0
edu                        0
edu_status              1569
edu_yr                  2756
equity                     0
f_birth                    0
f_edu                      0
f_political                0
f_work_14                  0
family_income              1
family_m                   0
family_status              0
floor_area                 0
gender                     0
health                     0
health_problem             0
height_cm                  0
house                      0
hukou                      0
hukou_loc                  4
id                         0
inc_ability                0
inc_exp                    0
income                     0
insur_1                    0
insur_2       

In [25]:
#数据的处理

#异常值，主要是对一些数据进行分箱，减少异常值的影响，有必要
#首先是日期处理，取出年份即可
data['survey_time'] = pd.to_datetime(data['survey_time'],format='%Y-%m-%d %H:%M:%S')
data["weekday"]=data["survey_time"].dt.weekday
data["year"]=data["survey_time"].dt.year
data["quarter"]=data["survey_time"].dt.quarter
data["hour"]=data["survey_time"].dt.hour
data["month"]=data["survey_time"].dt.month

data['age'] = data['year']-data['birth']
#data = data.drop(['survey_time','birth','weekday','year','quarter','hour','month','id'],axis=1)

In [26]:
#把一天的时间分段
def hour_cut(x):
    if 0<=x<6:
        return 0
    elif  6<=x<8:
        return 1
    elif  8<=x<12:
        return 2
    elif  12<=x<14:
        return 3
    elif  14<=x<18:
        return 4
    elif  18<=x<21:
        return 5
    elif  21<=x<24:
        return 6
    
data["hour_cut"]=data["hour"].map(hour_cut)

In [27]:
#调查年龄
data["survey_age"]=data["year"]-data["birth"]
#是否入党
data["join_party"]=data["join_party"].map(lambda x:0 if pd.isnull(x)  else 1)

#收入比
data['income/family_income'] = data['income']/(data['family_income']+1)
data['income/inc_exp'] = data['income']/(data['inc_exp']+1)
data['family_income/m'] = data['family_income']/(data['family_m']+0.01)

#收入/面积比
data['family_income/floor_area'] = data['family_income']/(data['floor_area']+0.01)

data['income/m'] = data['floor_area']/(data['family_m']+0.01)

#class
data['class_10_diff'] = (data['class_10_after'] - data['class_10_before'])
data['class_diff'] = data['class'] - data['class_10_before']
data['class_14_diff'] = data['class'] - data['class_14']


#province mean
data['province_income_mean'] = data.groupby(['province'])['income'].transform('mean').values
data['province_family_income_mean'] = data.groupby(['province'])['family_income'].transform('mean').values
data['province_equity_mean'] = data.groupby(['province'])['equity'].transform('mean').values
data['province_depression_mean'] = data.groupby(['province'])['depression'].transform('mean').values
data['province_floor_area_mean'] = data.groupby(['province'])['floor_area'].transform('mean').values

#city   mean
data['city_income_mean'] = data.groupby(['city'])['income'].transform('mean').values
data['city_family_income_mean'] = data.groupby(['city'])['family_income'].transform('mean').values
data['city_equity_mean'] = data.groupby(['city'])['equity'].transform('mean').values
data['city_depression_mean'] = data.groupby(['city'])['depression'].transform('mean').values
data['city_floor_area_mean'] = data.groupby(['city'])['floor_area'].transform('mean').values


#county  mean
data['county_income_mean'] = data.groupby(['county'])['income'].transform('mean').values
data['county_family_income_mean'] = data.groupby(['county'])['family_income'].transform('mean').values
data['county_equity_mean'] = data.groupby(['county'])['equity'].transform('mean').values
data['county_depression_mean'] = data.groupby(['county'])['depression'].transform('mean').values
data['county_floor_area_mean'] = data.groupby(['county'])['floor_area'].transform('mean').values

#ratio 相比同个城市
data['income/county'] = data['income']/(data['county_income_mean']+1)                                      
data['family_income/county'] = data['family_income']/(data['county_family_income_mean']+1)   
data['equity/county'] = data['equity']/(data['county_equity_mean']+1)       
data['depression/county'] = data['depression']/(data['county_depression_mean']+1)                                                
data['floor_area/county'] = data['floor_area']/(data['county_floor_area_mean']+1)   

#age   mean
data['age_income_mean'] = data.groupby(['survey_age'])['income'].transform('mean').values
data['age_family_income_mean'] = data.groupby(['survey_age'])['family_income'].transform('mean').values
data['age_equity_mean'] = data.groupby(['survey_age'])['equity'].transform('mean').values
data['age_depression_mean'] = data.groupby(['survey_age'])['depression'].transform('mean').values
data['age_floor_area_mean'] = data.groupby(['survey_age'])['floor_area'].transform('mean').values
data['age_health_mean'] = data.groupby(['survey_age','gender'])['health'].transform('mean').values
data['age_edu_mean'] = data.groupby(['survey_age','gender'])['edu'].transform('mean').values


#age/gender   mean
data['age_income_mean'] = data.groupby(['survey_age','gender'])['income'].transform('mean').values
data['age_family_income_mean'] = data.groupby(['survey_age','gender'])['family_income'].transform('mean').values
data['age_equity_mean'] = data.groupby(['survey_age','gender'])['equity'].transform('mean').values
data['age_depression_mean'] = data.groupby(['survey_age','gender'])['depression'].transform('mean').values
data['age_floor_area_mean'] = data.groupby(['survey_age','gender'])['floor_area'].transform('mean').values
# data['age_BMI_mean'] = data.groupby(['survey_age','gender'])['BMI'].transform('mean').values
data['age_gender_health_mean'] = data.groupby(['survey_age','gender'])['health'].transform('mean').values


#class   mean
data['city_class_income_mean'] = data.groupby(['class'])['income'].transform('mean').values
data['city_class_family_income_mean'] = data.groupby(['class'])['family_income'].transform('mean').values
data['city_class_equity_mean'] = data.groupby(['class'])['equity'].transform('mean').values
data['city_class_depression_mean'] = data.groupby(['class'])['depression'].transform('mean').values
data['city_class_floor_area_mean'] = data.groupby(['class'])['floor_area'].transform('mean').values



In [28]:
#出生的年代
def birth_split(x):
    if 1920<=x<=1930:
        return 0
    elif  1930<x<=1940:
        return 1
    elif  1940<x<=1950:
        return 2
    elif  1950<x<=1960:
        return 3
    elif  1960<x<=1970:
        return 4
    elif  1970<x<=1980:
        return 5
    elif  1980<x<=1990:
        return 6
    elif  1990<x<=2000:
        return 7
    
data["birth_s"]=data["birth"].map(birth_split)

In [29]:
#收入分组
def income_cut(x):
    if x<0:
        return 0
    elif  0<=x<1200:
        return 1
    elif  1200<x<=10000:
        return 2
    elif  10000<x<24000:
        return 3
    elif  24000<x<40000:
        return 4
    elif  40000<=x:
        return 5
 
data["income_cut"]=data["income"].map(income_cut)

In [30]:
##缺失值填充 
data['edu_status'] = data['edu_status'].fillna(5)
data["edu_yr"]=data["edu_yr"].fillna(-2)
data['social_neighbor'] = data['social_neighbor'].fillna(8)
data['social_friend'] = data['social_friend'].fillna(8)
data["work_status"]=data["work_status"].fillna(0)
data["work_yr"]=data["work_yr"].fillna(0)
data["work_type"]=data["work_type"].fillna(0)
data["work_manage"]=data["work_manage"].fillna(0)
data["family_income"]=data["family_income"].fillna(-2)
data["minor_child"]=data["minor_child"].fillna(0)
data["marital_1st"]=data["marital_1st"].fillna(0)
data["s_birth"]=data["s_birth"].fillna(0)
data["marital_now"]=data["marital_now"].fillna(0)
data["s_edu"]=data["s_edu"].fillna(0)
data["s_political"]=data["s_political"].fillna(0)
data["s_hukou"]=data["s_hukou"].fillna(0)
data["s_income"]=data["s_income"].fillna(0)
data["s_work_exper"]=data["s_work_exper"].fillna(0)
data["s_work_status"]=data["s_work_status"].fillna(0)
data["s_work_type"]=data["s_work_type"].fillna(0)
data["hukou_loc"]=data["hukou_loc"].fillna(4)
data = data.drop(['survey_time'],axis=1)
data.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10968 entries, 0 to 2967
Data columns (total 186 columns):
birth                            10968 non-null int64
car                              10968 non-null int64
city                             10968 non-null int64
class                            10968 non-null int64
class_10_after                   10968 non-null int64
class_10_before                  10968 non-null int64
class_14                         10968 non-null int64
county                           10968 non-null int64
daughter                         10968 non-null int64
depression                       10968 non-null int64
edu                              10968 non-null int64
edu_status                       10968 non-null float64
edu_yr                           10968 non-null float64
equity                           10968 non-null int64
f_birth                          10968 non-null int64
f_edu                            10968 non-null int64
f_political            

In [31]:
data.head()

,birth,car,city,class,class_10_after,class_10_before,class_14,county,daughter,depression,edu,edu_status,edu_yr,equity,f_birth,f_edu,f_political,f_work_14,family_income,family_m,family_status,floor_area,gender,health,health_problem,height_cm,house,hukou,hukou_loc,id,inc_ability,inc_exp,income,insur_1,insur_2,insur_3,insur_4,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,join_party,learn,leisure_1,leisure_10,leisure_11,leisure_12,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,m_birth,m_edu,m_political,m_work_14,marital,marital_1st,marital_now,media_1,media_2,media_3,media_4,media_5,media_6,minor_child,nationality,neighbor_familiarity,political,property_0,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,province,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9,relax,religion,religion_freq,s_birth,s_edu,s_hukou,s_income,s_political,s_work_exper,s_work_status,s_work_type,socia_outing,social_friend,social_neighbor,socialize,son,status_3_before,status_peer,survey_type,trust_1,trust_10,trust_11,trust_12,trust_13,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,view,weight_jin,work_exper,work_manage,work_status,work_type,work_yr,weekday,year,quarter,hour,month,age,hour_cut,survey_age,income/family_income,income/inc_exp,family_income/m,family_income/floor_area,income/m,class_10_diff,class_diff,class_14_diff,province_income_mean,province_family_income_mean,province_equity_mean,province_depression_mean,province_floor_area_mean,city_income_mean,city_family_income_mean,city_equity_mean,city_depression_mean,city_floor_area_mean,county_income_mean,county_family_income_mean,county_equity_mean,county_depression_mean,county_floor_area_mean,income/county,family_income/county,equity/county,depression/county,floor_area/county,age_income_mean,age_family_income_mean,age_equity_mean,age_depression_mean,age_floor_area_mean,age_health_mean,age_edu_mean,age_gender_health_mean,city_class_income_mean,city_class_family_income_mean,city_class_equity_mean,city_class_depression_mean,city_class_floor_area_mean,birth_s,income_cut
0,1959,2,32,3,3,3,1,59,0,5,11,4.0,-2.0,3,-2,4,4,1,60000.0,2,2,45.0,1,3,2,176,1,5,2.0,1,3,50000.0,20000,1,1,1,2,0,1,0,0,0,0,0,0,0,0,3,1,5,4,1,4,3,1,2,3,4,1,4,-2,4,1,1,3,1984.0,1984.0,4,2,5,5,4,3,0.0,1,4,1,0,1,0,0,0,0,0,0,0,12,50,60.0,50,50,30.0,30,50,50,50,4,1,1,1958.0,6.0,5.0,40000.0,1.0,5.0,0.0,0.0,2,3.0,3.0,2,1,2,3,1,4,3,-8,4,1,2,-8,-8,5,3,2,3,4,4,155,1,2.0,3.0,1.0,30.0,1,2015,3,14,8,56,4,56,0.333328,0.399992,29850.746269,1333.037103,22.388060,0,0,2,61276.734463,117277.474576,2.612053,3.926554,88.676271,42968.202381,85781.863095,2.758929,4.008929,82.647321,28979.469388,56567.204082,2.571429,4.081633,38.530612,0.690120,1.060667,0.840000,0.983936,1.138358,29988.261538,66990.553846,3.046154,3.907692,96.089231,3.476923,5.169231,3.476923,24608.028533,39439.561821,2.978261,3.648098,107.182948,3,3.0
1,1992,2,52,6,8,4,5,85,0,3,12,4.0,2013.0,3,1972,3,1,2,40000.0,3,4,110.0,1,5,4,170,1,1,1.0,2,2,50000.0,20000,1,1,1,1,0,1,0,0,0,0,0,0,0,0,3,2,4,5,1,3,4,3,5,4,3,2,3,1973,3,1,2,1,0.0,0.0,2,2,1,3,5,1,0.0,1,3,1,0,0,0,0,1,0,0,0,0,18,90,70.0,70,80,85.0,70,90,60,60,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,6.0,2,0,1,1,2,5,3,3,3,2,4,4,3,5,3,3,3,2,4,110,1,3.0,3.0,1.0,2.0,1,2015,3,15,7,23,4,23,0.499988,0.399992,13289.036545,363.603309,36.544850,4,2,1,29806.161512,33767.848797,3.207904,3.991409,148.463918,64016.052356,30031.596859,3.308901,3.989529,157.528796,11628.587629,29969.742268,3.350515,3.958763,154.608247,1.719751,1.334635,0.689573,0.604990,0.706903,25553.492754,62796.681159,3.231884,4.115942,103.115942,4.202899,9.101449,4.202899,43439.878453,104062.348066,3.312549,4.023678,124.915075,7,3.0
2,1967,2,83,5,6,4,3,126,2,5,4,4.0,-2.0,4,-2,1,1,2,8000.0,3,3,120.0,2,4,4,160,1,1,1.0,3,2,80000.0,2000,1,1,2,2,0,1,0,0,0,0,0,0,0,0,2,1,5,5,5,

In [32]:
tests = data.iloc[8000:,:]
data_drop_part = data.iloc[:8000,:]
#data_drop_part.info(verbose = True,null_counts=True)
#tests.value_counts()

In [33]:
#使用所有特征
##接下来是XGB的回归
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, RepeatedKFold
from scipy import sparse
from sklearn.model_selection import cross_val_score as CVS,train_test_split as TTS

#自定义评价函数
def myFeval(preds, xgbtrain):
    label = xgbtrain.get_label()
    score = mean_squared_error(label,preds)
    return 'myFeval',score

xgb_params = {'n_estimators':100,
              "booster":'gbtree','eta': 0.005, 
              'max_depth': 5, 'subsample': 0.8, 
              'colsample_bytree': 0.3, 
              'objective': 'reg:squarederror', 
              'eval_metric': 'rmse'
              }

folds = KFold(n_splits=5, shuffle=True, random_state=2018)

X_train = data_drop_part.reset_index(drop=True).values
y_train = y_labels.reset_index(drop=True).values.reshape(8000,1)
test = tests.reset_index(drop=True).values
oof_xgb = np.zeros(len(X_train))
predictions_xgb = np.zeros(len(test))
                                              
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=2940, evals=watchlist, early_stopping_rounds=200, 
                    verbose_eval=100, params=xgb_params,feval = myFeval)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train)))

#初始结果0.45

[22:55:07] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:3.45059	train-myFeval:11.90659	valid_data-rmse:3.43890	valid_data-myFeval:11.82601
[100]	train-rmse:2.16849	train-myFeval:4.70236	valid_data-rmse:2.15801	valid_data-myFeval:4.65699
[200]	train-rmse:1.42242	train-myFeval:2.02328	valid_data-rmse:1.41844	valid_data-myFeval:2.01198
[300]	train-rmse:1.00781	train-myFeval:1.01568	valid_data-rmse:1.01669	valid_data-myFeval:1.03367
[400]	train-rmse:0.79248	train-myFeval:0.62803	valid_data-rmse:0.81838	valid_data-myFeval:0.66975
[500]	train-rmse:0.68734	train-myFeval:0.47243	valid_data-rmse:0.73073	valid_data-myFeval:0.53397
[600]	train-rmse:0.63575	train-myFeval:0.40417	valid_data-rmse:0.69412	valid_dat

In [34]:
#xgb的网格调参
## 从sklearn库中导入网格调参函数
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

#先进行随机调参
parameters = {
        'n_estimators':range(80,200,4),
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20),
        'colsample_bytree':np.linspace(0.5,0.98,10),
        'min_child_weight':range(1,9,1)
        }

model = xgb.XGBRFRegressor()

## 进行网格搜索
clf = RandomizedSearchCV(model, parameters, cv=5, scoring='neg_root_mean_squared_error',verbose=1,n_jobs=-1)
clf = clf.fit(X_train, y_train)

clf.best_params_




Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'subsample': 0.7210526315789473,
 'n_estimators': 124,
 'min_child_weight': 6,
 'max_depth': 10,
 'learning_rate': 0.8478947368421053,
 'colsample_bytree': 0.7133333333333334}

In [35]:
##网格调参
## 定义参数取值范围
learning_rate = [0.005, 0.007,0.009,0.01]
subsample = [ 0.8, 0.85, 0.9, 0.95]
colsample_bytree = [0.8,0.85,0.9,0.95]
max_depth = [5,6,7]
n_estimators = [1000,2000,3000,4000,5000]
min_child_weight = [3,4,5]


parameters = {'learning_rate': learning_rate, 
              'max_depth': max_depth, 
              'subsample': subsample, 
              'colsample_bytree': colsample_bytree, 
              'min_child_weight': min_child_weight}
model = xgb.XGBRFRegressor()

## 进行网格搜索
clf = GridSearchCV(model, parameters, cv=5, scoring='neg_root_mean_squared_error',verbose=1,n_jobs=-1)
clf = clf.fit(X_train, y_train)

clf.best_params_

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


{'colsample_bytree': 0.95,
 'learning_rate': 0.01,
 'max_depth': 7,
 'min_child_weight': 5,
 'subsample': 0.8}